In [ ]:
import re
from nltk.tokenize import sent_tokenize, word_tokenize
import math

# Function to read transcript from a file
def read_transcript(file_path):
    with open(file_path, 'r') as f:
        return f.read()

# Function to check for matching sentences or consecutive words
def check_match(segment, clip, min_words=8):
    for sentence in clip:
        if len(word_tokenize(sentence)) >= min_words:
            if sentence in segment:
                return True, [sentence]
    
    for sentence in segment:
        segment_words = word_tokenize(sentence)
        for clip_sentence in clip:
            clip_words = word_tokenize(clip_sentence)
            for i in range(len(segment_words) - min_words + 1):
                if ' '.join(segment_words[i:i + min_words]) in ' '.join(clip_words):
                    return True, [' '.join(segment_words[i:i + min_words])]
    return False, []

# Function to label segments
def label_segments(youtube_file, tiktok_files):
    # Read YouTube transcript
    youtube_transcript = read_transcript(youtube_file)
    youtube_sentences = sent_tokenize(youtube_transcript)
    
    # Read TikTok transcripts
    tiktok_clips = []
    for file in tiktok_files:
        tiktok_transcript = read_transcript(file)
        tiktok_sentences = sent_tokenize(tiktok_transcript)
        tiktok_clips.append(tiktok_sentences)
    
    # Segment YouTube transcript
    segment_length = math.ceil(len(youtube_sentences) / 60)  # Adjust as needed
    segments = [youtube_sentences[i:i + segment_length] for i in range(0, len(youtube_sentences), segment_length)]
    
    # Label segments
    labels = []
    for segment in segments:
        label = 0
        matching_sentences = []
        for clip in tiktok_clips:
            match, match_sentences = check_match(segment, clip)
            if match:
                label = 1
                matching_sentences.extend(match_sentences)
                break
        labels.append(label)
        
        # Print the matching sentences if label is 1
        if label == 1:
            print(f"Matching sentences: {matching_sentences}")
    
    # Output labeled segments
    labeled_segments = list(zip(segments, labels))
    return labeled_segments

# File paths
youtube_file = "/Users/rossjackson/Documents/VideoAnalysisProject/New_Project_Directory/TikTok:YT_Audios/#24/#24 Trust ｜ Cheating Partners, Death, Knife crime and being let down.txt"
tiktok_files = [
    "/Users/rossjackson/Documents/VideoAnalysisProject/New_Project_Directory/TikTok:YT_Audios/#24/yt_24_1.txt",
    "/Users/rossjackson/Documents/VideoAnalysisProject/New_Project_Directory/TikTok:YT_Audios/#24/yt_24_2.txt",
    "/Users/rossjackson/Documents/VideoAnalysisProject/New_Project_Directory/TikTok:YT_Audios/#24/yt_24_3.txt",
    "/Users/rossjackson/Documents/VideoAnalysisProject/New_Project_Directory/TikTok:YT_Audios/#24/yt_24_4.txt",
    "/Users/rossjackson/Documents/VideoAnalysisProject/New_Project_Directory/TikTok:YT_Audios/#24/yt_24_5.txt"
]

# Label the segments
labeled_segments = label_segments(youtube_file, tiktok_files)

# Print out some labeled segments for verification
for i, (segment, label) in enumerate(labeled_segments):  # Adjust to see more or fewer segments
    print(f"Segment {i+1} (Label: {label}):\n{' '.join(segment)}\n")
